In [8]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

import datetime


from pytz import timezone
from pycaret.classification import *

warnings.filterwarnings('ignore')

In [9]:
def get_db_connection(dbEnvironment):
    if dbEnvironment =='QA':
        try:
            # connect to SQL Server.
            SERVER = '52.44.171.130' 
            DATABASE = 'datascience' 
            USERNAME = 'nrad' 
            PASSWORD = 'ThisIsQA123' #getpass.getpass(prompt='Enter your password')
            CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
            print("The Connection to "+dbEnvironment +" was succesfull")
        except Exception as err:
            print("The Connection to "+dbEnvironment +" wasn't succesfull", err)
    elif dbEnvironment =='PRD':
        try:
            # connect to SQL Server.
            SERVER = '34.206.73.189' 
            DATABASE = 'datascience' 
            USERNAME = 'nrad' 
            PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
            CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
            print("The Connection to "+dbEnvironment +" was succesfull")
        except Exception as err:
            print("The Connection to "+dbEnvironment +" wasn't succesfull", err)
    return CNXN
            

In [10]:
def get_datetime():
    today = datetime.datetime.now()
    todaydate = today.strftime("%Y-%m-%d")
    todaydate_id = today.strftime("%Y%m%d")
    return todaydate
    

In [14]:
def get_ds_scores(connection,datetimeparam):
    try:
        
        #today = datetime.datetime.now()
        #today_date = today.strftime("%Y-%m-%d")
        cursor = connection.cursor()
        clientid =  f'''select distinct lkupclientid, stlrDBName from ds.productyear_all;'''
        clientid_df = pd.read_sql(clientid, connection)
        lkupclientid = clientid_df['lkupclientid']
        stlrdbname = clientid_df['stlrDBName']
        for idx, m in enumerate(lkupclientid):
            lkupclientid_value=lkupclientid[idx]
            stlrdb_value= stlrdbname[idx]
            numRecords =  f'''select count(*) as numRecords  from ds.customerScores where lkupclientid ={lkupclientid_value} and cast(insertDate as date) = {"'"+ str(datetimeparam) + "'"} group by lkupclientid;'''
            cntTodayScores = pd.read_sql(numRecords, connection)
            cntrecords= cntTodayScores['numRecords']
            for idx, m in enumerate(cntrecords):
                value=cntrecords[idx]
                if value == 0:
                    print("ds.customerScores is not updated for " +str(lkupclientid_value))
                elif value !=0:
                    print("ds.customerScores table is updated and " + str(value) + " records been inserted on " + str(datetimeparam) + " for "+ str(lkupclientid_value))
        connection.commit()
        cursor.close()
    except Exception as err:
            print("The Connection wasn't succesfull", err)



In [5]:
def get_stlr_scores(factconnection):
    try:
        
        today = datetime.datetime.now()
        today_date_id = today.strftime("%Y%m%d")
        cursor = factconnection.cursor()
        clientid =  f'''select distinct lkupclientid, stlrDBName from ds.productyear_all;'''
        clientid_df = pd.read_sql(clientid, factconnection)
        lkupclientid = clientid_df['lkupclientid']
        stlrdbname = clientid_df['stlrDBName']
        for idx, m in enumerate(lkupclientid):
            lkupclientid_value=lkupclientid[idx]
            stlrdb_value= stlrdbname[idx] 
    
            try:
                SERVER = '34.206.73.189'
                DATABASE = stlrdb_value 
                USERNAME = 'nrad' 
                PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
                CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
                print("The Connection to was succesfull to "+  stlrdb_value)
                fact_numRecords =  f'''select count(*) as factnumRecords from dw.factCustomerretentionScore where lkupclientid ={lkupclientid_value} and cast(dimDateId as int) = {today_date_id};'''
                cntTodayScores = pd.read_sql(fact_numRecords, CNXN)
                cntrecords= cntTodayScores['factnumRecords']
                for idx, m in enumerate(cntrecords):
                    value=cntrecords[idx]
                    if value == 0:
                        print("dw.factCustomerretentionScore is not updated for " +str(lkupclientid_value))
                    elif value !=0:
                        print("dw.factCustomerretentionScore table is updated and " + str(value) + " records been inserted on " + str(today_date_id) + " for "+ str(lkupclientid_value))
            except Exception as err:
                print("The Connection to wasn't succesfull to " + stlrdb_value, err)
        factconnection.commit()
        cursor.close()
    except Exception as err:
            print("The Connection wasn't succesfull", err)

In [15]:
def main():
    
    conn = get_db_connection("PRD")
    today_date = get_datetime()
    retention_scores = get_ds_scores(conn,today_date)
    #fact_retention_scores = get_stlr_scores(conn)
if __name__ == "__main__":
    main()



The Connection to PRD was succesfull
